# Crie um aplicativo LLM simples com LCEL

Para instalar o LangChain, execute:

In [5]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


##### LangSmith

Muitas das aplicações que você constrói com LangChain conterão múltiplas etapas com várias invocações de chamadas de LLM. À medida que essas aplicações se tornam mais complexas, torna-se crucial poder inspecionar o que exatamente está acontecendo dentro da sua cadeia ou agente. A melhor maneira de fazer isso é com LangSmith.

Após se inscrever no link acima, certifique-se de configurar suas variáveis de ambiente para começar a registrar rastreamentos:

export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."

Ou:

In [6]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


##### Usando Modelos de Linguagem

Primeiro, vamos aprender como usar um modelo de linguagem por si só. LangChain suporta diversos modelos de linguagem que podem ser usados de forma intercambiável
selecione abaixo o que você deseja utilizar!

In [9]:
pip install langchain-mistralai

Note: you may need to restart the kernel to use updated packages.


In [10]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = getpass.getpass()

from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")

 ········


Vamos primeiro usar o modelo diretamente. ChatModels são instâncias de "Runnables" do LangChain, o que significa que eles expõem uma interface padrão para interagir com eles. Para simplesmente chamar o modelo, podemos passar uma lista de mensagens para o método .invoke.

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into portuguese"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Hello! In Portuguese, this is "Olá!" Please note that, like in English, the exclamation mark is used to convey enthusiasm or excitement.', response_metadata={'token_usage': {'prompt_tokens': 17, 'total_tokens': 50, 'completion_tokens': 33}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-d022b7ff-5534-4dd6-86df-1a41159da4c8-0')


Se tivermos habilitado o LangSmith, podemos ver que esta execução é registrada no LangSmith e podemos ver o rastreamento no LangSmith.

##### OutputParsers

Observe que a resposta do modelo é um AIMessage. Isso contém uma resposta em forma de string junto com outros metadados sobre a resposta. Muitas vezes, podemos querer trabalhar apenas com a resposta em forma de string. Podemos extrair apenas esta resposta utilizando um parser de saída simples.

Primeiro, importamos o parser de saída simples.

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

Uma forma de usá-lo é usá-lo sozinho. Por exemplo, poderíamos salvar o resultado da chamada do modelo de linguagem e então passá-lo para o parser.

In [13]:
result = model.invoke(messages)

In [14]:
parser.invoke(result)

'Hello! In Portuguese, this is typically translated as "Olá!" Please note that, like in English, there are many ways to greet someone in Portuguese, and the choice often depends on the context, formality, and regional variations of the language.'

Mais comumente, podemos "encadear" o modelo com este parser de saída. Isso significa que este parser de saída será chamado toda vez nesta cadeia. Esta cadeia assume o tipo de entrada do modelo de linguagem (string ou lista de mensagens) e retorna o tipo de saída do parser de saída (string).

Podemos criar facilmente a cadeia usando o operador |. O operador | é usado no LangChain para combinar dois elementos.

In [15]:
chain = model | parser

In [16]:
chain.invoke(messages)

'Hello! In Portuguese, this is typically translated as "Olá!" Please note that, like in English, there are many ways to greet someone in Portuguese, and the choice often depends on the context and the relationship between the people conversing. For example, "Oi" is another common, informal way to say "hi" in Portuguese.'

##### Templates de Prompt

Atualmente, estamos passando uma lista de mensagens diretamente no modelo de linguagem. De onde vem essa lista de mensagens? Normalmente, ela é construída a partir de uma combinação de entrada do usuário e lógica de aplicação. Essa lógica de aplicação geralmente pega a entrada bruta do usuário e a transforma em uma lista de mensagens pronta para passar ao modelo de linguagem. Transformações comuns incluem adicionar uma mensagem de sistema ou formatar um template com a entrada do usuário.

PromptTemplates são um conceito no LangChain projetado para auxiliar nessa transformação. Eles recebem a entrada bruta do usuário e retornam dados (um prompt) que estão prontos para serem passados para um modelo de linguagem.

Vamos criar um PromptTemplate aqui. Ele receberá duas variáveis do usuário:

- language: O idioma para o qual traduzir o texto
- text: O texto a ser traduzido

In [17]:
from langchain_core.prompts import ChatPromptTemplate

Primeiro, vamos criar uma string que formataremos para ser a mensagem do sistema:

In [18]:
system_template = "Translate the following into {language}:"

A seguir, podemos criar o PromptTemplate. Este será uma combinação do system_template, bem como um modelo mais simples para onde colocar o texto.

In [19]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

A entrada para este modelo de prompt é um dicionário. Podemos brincar com este modelo de prompt por si só para ver o que ele faz por si mesmo.

In [21]:
result = prompt_template.invoke({"language": "portuguese", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into portuguese:'), HumanMessage(content='hi')])

Podemos ver que ele retorna um ChatPromptValue que consiste de dois mensagens. Se quisermos acessar as mensagens diretamente, fazemos:

In [23]:
result.to_messages()

[SystemMessage(content='Translate the following into portuguese:'),
 HumanMessage(content='hi')]

##### Encadeando componentes com LCEL

Agora podemos combinar isso com o modelo e o analisador de saída mencionados acima usando o operador pipe (|):

In [25]:
chain = prompt_template | model | parser

In [27]:
chain.invoke({"language": "portuguese", "text": "hi"})

'Hello! The Portuguese translation for "hi" is "olá". So, you can say "olá" to greet someone in Portuguese.'

Este é um exemplo simples de como usar a Linguagem de Expressão LangChain (LCEL) para encadear módulos LangChain. Existem várias vantagens neste método, incluindo otimização de streaming e suporte a rastreamento.

Se observarmos o rastreamento do LangSmith, podemos ver que todos os três componentes aparecem no rastreamento do LangSmith.